In [1]:
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime

In [2]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe'

In [3]:
#맞춰서 수정필요
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
search = "풀무원 바른먹거리"

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
options = webdriver.ChromeOptions()
option = Options()
options.add_argument("headless")
option.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 1
})
usr = "아이디"
pwd = "비밀번호"

def facebook_crwal(search,driver_path):
    posts_list = []
    comments_list = []
    driver = webdriver.Chrome(options=option, executable_path=driver_path) #팝업만제거, 창제거까지 원하면 options로
    driver.get("http://www.facebook.org")
    assert "Facebook" in driver.title
    elem = driver.find_element_by_id("email")
    elem.send_keys(usr)
    elem = driver.find_element_by_id("pass")
    elem.send_keys(pwd)
    elem.send_keys(Keys.RETURN)
    time.sleep(5)

    #게시물 검색
    elem = driver.find_element_by_name("global_typeahead")
    elem.send_keys(search)
    time.sleep(2)
    elem.send_keys(Keys.RETURN)
    time.sleep(3)
    elem = driver.find_element_by_link_text('게시물')
    elem.send_keys(Keys.RETURN)
    time.sleep(3)
    elem = driver.find_element_by_class_name('plgsh5y4.pmk7jnqg.ditlmg2l.dpjh1vo5')
    elem.click()
    elem = driver.find_element_by_xpath('//*[@id="전체 공개 게시물"]/div/div[1]/div/div/div/span/span')
    elem.click()
    time.sleep(3)

    #맨 아래까지 스크롤해서 데이터 불러오기
    prev_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 스크롤을 화면 가장 아래로 내린다
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")    
        # 페이지 로딩 대기
        time.sleep(1)
        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")

        if(curr_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")
  #####여기까지는 문제X#################        
        
    url_domain = 'https://www.facebook.com/search/posts?q='
    filter_domain ='&filters=eyJycF9hdXRob3I6MCI6IntcIm5hbWVcIjpcIm1lcmdlZF9wdWJsaWNfcG9zdHNcIixcImFyZ3NcIjpcIlwifSJ9' 
    url = url_domain + search + filter_domain
    soup = BeautifulSoup(url,'html.parser')

    try:
        read_list = soup.select('div', class_='d2edcug0 o7dlgrpb')
        read_list = read_list[0].find_all('div', class_='rq0escxv l9j0dhe7 du4w35lb j83agx80 cbu4d94t pfnyh3mw d2edcug0')
        url_list = []
        for url in read_list:
            url_list.append(str(url.find_all('a')[0].get('href')))
    except:
        print('결과가 없습니다')
        driver.close()
        return 0
    driver.close()

    for url in url_list:
        try:
            driver = webdriver.Chrome(executable_path=driver_path, options=option)  # for Windows
            driver.get(url)
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source,'html.parser')
            posts = clean_text(soup.find('#jsc_c_1h > div').text)
            posts_list.append(post)
            comment = soup.select_one('div', class_='cwj9oz12 tvmv18p').find_all('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql')
            comments = [clean_text(x.text) for x in comment]
            comments_list.append(' '.join(comments))
        except:
            print("None_type")
        driver.close()
    output = pd.DataFrame({'기업 슬로건' : [search]*len(posts_list),
                               '게시물' :  posts_list,
                               '댓글' : comments_list})
    return output

facebook = facebook_crwal(search,driver_path)
    

결과가 없습니다


In [15]:
facebook.to_csv('페이스북.csv',encoding='UTF8',index=None)

AttributeError: 'int' object has no attribute 'to_csv'

In [ ]:
test = pd.read_csv('페이스북.csv',encoding='UTF8')

In [ ]:
test

'#'이 앞에 붙는 단어들 따로 빼서 표본에 넣을지 말지 판단
댓글 '더보기'로 더 로드해야하는지 결과보고 필요시 재설정
중복제거